In [2]:
import keras
import numpy as np
from keras.models import Sequential
from keras.layers import Dense, LSTM, Dropout
from sklearn.preprocessing import MinMaxScaler
import matplotlib.pyplot as plt
%matplotlib inline
import random

Using TensorFlow backend.


In [3]:
# 1. 데이터셋 생성하기
signal_data = []
for i in range(365):
    random_5 = random.randint(1,5)
    signal_data.append(random_5)
signal_data=np.array(signal_data)
signal_data=signal_data[:,None]

In [4]:
print(signal_data)

[[1]
 [2]
 [1]
 [4]
 [1]
 [2]
 [3]
 [1]
 [5]
 [5]
 [2]
 [3]
 [4]
 [5]
 [5]
 [2]
 [4]
 [4]
 [2]
 [4]
 [2]
 [1]
 [4]
 [2]
 [2]
 [1]
 [5]
 [3]
 [2]
 [5]
 [2]
 [3]
 [3]
 [5]
 [3]
 [3]
 [1]
 [4]
 [4]
 [3]
 [3]
 [1]
 [3]
 [1]
 [4]
 [2]
 [1]
 [5]
 [2]
 [2]
 [1]
 [3]
 [2]
 [1]
 [5]
 [3]
 [5]
 [2]
 [1]
 [2]
 [5]
 [3]
 [1]
 [5]
 [5]
 [5]
 [5]
 [5]
 [5]
 [1]
 [5]
 [3]
 [5]
 [4]
 [2]
 [5]
 [4]
 [5]
 [1]
 [4]
 [3]
 [1]
 [4]
 [2]
 [3]
 [3]
 [2]
 [3]
 [1]
 [5]
 [3]
 [3]
 [4]
 [4]
 [4]
 [1]
 [3]
 [2]
 [5]
 [1]
 [3]
 [2]
 [3]
 [5]
 [1]
 [2]
 [4]
 [3]
 [3]
 [1]
 [3]
 [1]
 [5]
 [3]
 [2]
 [3]
 [2]
 [1]
 [1]
 [1]
 [4]
 [3]
 [5]
 [4]
 [1]
 [1]
 [3]
 [4]
 [1]
 [2]
 [3]
 [4]
 [3]
 [2]
 [4]
 [2]
 [3]
 [5]
 [2]
 [2]
 [5]
 [1]
 [1]
 [3]
 [5]
 [4]
 [2]
 [1]
 [2]
 [5]
 [3]
 [4]
 [4]
 [2]
 [1]
 [5]
 [1]
 [5]
 [1]
 [5]
 [5]
 [5]
 [4]
 [5]
 [1]
 [5]
 [1]
 [4]
 [1]
 [1]
 [1]
 [5]
 [4]
 [2]
 [4]
 [4]
 [1]
 [2]
 [3]
 [4]
 [5]
 [5]
 [1]
 [2]
 [2]
 [4]
 [2]
 [2]
 [1]
 [2]
 [4]
 [4]
 [5]
 [2]
 [2]
 [4]
 [1]
 [4]
 [2]
 [4]


In [11]:
scaler = MinMaxScaler(feature_range=(1, 100))
signal_data = scaler.fit_transform(signal_data)

In [12]:
print(signal_data)

[[  1.  ]
 [ 25.75]
 [  1.  ]
 [ 75.25]
 [  1.  ]
 [ 25.75]
 [ 50.5 ]
 [  1.  ]
 [100.  ]
 [100.  ]
 [ 25.75]
 [ 50.5 ]
 [ 75.25]
 [100.  ]
 [100.  ]
 [ 25.75]
 [ 75.25]
 [ 75.25]
 [ 25.75]
 [ 75.25]
 [ 25.75]
 [  1.  ]
 [ 75.25]
 [ 25.75]
 [ 25.75]
 [  1.  ]
 [100.  ]
 [ 50.5 ]
 [ 25.75]
 [100.  ]
 [ 25.75]
 [ 50.5 ]
 [ 50.5 ]
 [100.  ]
 [ 50.5 ]
 [ 50.5 ]
 [  1.  ]
 [ 75.25]
 [ 75.25]
 [ 50.5 ]
 [ 50.5 ]
 [  1.  ]
 [ 50.5 ]
 [  1.  ]
 [ 75.25]
 [ 25.75]
 [  1.  ]
 [100.  ]
 [ 25.75]
 [ 25.75]
 [  1.  ]
 [ 50.5 ]
 [ 25.75]
 [  1.  ]
 [100.  ]
 [ 50.5 ]
 [100.  ]
 [ 25.75]
 [  1.  ]
 [ 25.75]
 [100.  ]
 [ 50.5 ]
 [  1.  ]
 [100.  ]
 [100.  ]
 [100.  ]
 [100.  ]
 [100.  ]
 [100.  ]
 [  1.  ]
 [100.  ]
 [ 50.5 ]
 [100.  ]
 [ 75.25]
 [ 25.75]
 [100.  ]
 [ 75.25]
 [100.  ]
 [  1.  ]
 [ 75.25]
 [ 50.5 ]
 [  1.  ]
 [ 75.25]
 [ 25.75]
 [ 50.5 ]
 [ 50.5 ]
 [ 25.75]
 [ 50.5 ]
 [  1.  ]
 [100.  ]
 [ 50.5 ]
 [ 50.5 ]
 [ 75.25]
 [ 75.25]
 [ 75.25]
 [  1.  ]
 [ 50.5 ]
 [ 25.75]
 [100.  ]
 [  1.  ]


In [13]:
train = signal_data[0:290]
test = signal_data[290:]

In [16]:
def create_dataset(signal_data, look_back=1):
    dataX, dataY = [], []
    for i in range(len(signal_data)-look_back):
        dataX.append(signal_data[i:(i+look_back), 0])
        dataY.append(signal_data[i + look_back, 0])
    return np.array(dataX), np.array(dataY)

class CustomHistory(keras.callbacks.Callback):
    def init(self):
        self.train_loss = []
        self.val_loss = []
        
    def on_epoch_end(self, batch, logs={}):
        self.train_loss.append(logs.get('loss'))
        self.val_loss.append(logs.get('val_loss'))

In [30]:
x_train, y_train = create_dataset(train, 10)
x_test, y_test = create_dataset(test, 10)

In [28]:
len(y_train)

280

In [32]:
len(x_test)

65

In [17]:
a = []
b = []

In [18]:
data_lst = list(range(20))
print(data_lst)

[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19]


In [40]:
signal_data

array([[  1.  ],
       [ 25.75],
       [  1.  ],
       [ 75.25],
       [  1.  ],
       [ 25.75],
       [ 50.5 ],
       [  1.  ],
       [100.  ],
       [100.  ],
       [ 25.75],
       [ 50.5 ],
       [ 75.25],
       [100.  ],
       [100.  ],
       [ 25.75],
       [ 75.25],
       [ 75.25],
       [ 25.75],
       [ 75.25],
       [ 25.75],
       [  1.  ],
       [ 75.25],
       [ 25.75],
       [ 25.75],
       [  1.  ],
       [100.  ],
       [ 50.5 ],
       [ 25.75],
       [100.  ],
       [ 25.75],
       [ 50.5 ],
       [ 50.5 ],
       [100.  ],
       [ 50.5 ],
       [ 50.5 ],
       [  1.  ],
       [ 75.25],
       [ 75.25],
       [ 50.5 ],
       [ 50.5 ],
       [  1.  ],
       [ 50.5 ],
       [  1.  ],
       [ 75.25],
       [ 25.75],
       [  1.  ],
       [100.  ],
       [ 25.75],
       [ 25.75],
       [  1.  ],
       [ 50.5 ],
       [ 25.75],
       [  1.  ],
       [100.  ],
       [ 50.5 ],
       [100.  ],
       [ 25.75],
       [  1.  

In [42]:
signal_data[1,0]

25.75

In [43]:
signal_data[0:10,0]

array([  1.  ,  25.75,   1.  ,  75.25,   1.  ,  25.75,  50.5 ,   1.  ,
       100.  , 100.  ])

In [44]:
a.append(signal_data[0:10,0])

In [45]:
a

[array([  1.  ,  25.75,   1.  ,  75.25,   1.  ,  25.75,  50.5 ,   1.  ,
        100.  , 100.  ])]

In [46]:
a.append(signal_data[1:11,0])

In [47]:
a

[array([  1.  ,  25.75,   1.  ,  75.25,   1.  ,  25.75,  50.5 ,   1.  ,
        100.  , 100.  ]),
 array([ 25.75,   1.  ,  75.25,   1.  ,  25.75,  50.5 ,   1.  , 100.  ,
        100.  ,  25.75])]